# Imports

In [1]:
import torch
import torch.nn as nn
from utils.datautils import *
from utils.MLutils import *
from utils.resources import *
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import BertModel
import unicodedata

# linea que arregla algunos errores de loadeo de datasets
# pip install --upgrade datasets

/home/fabro/Documents/uba/aprendizaje_automatico/Puntualizacion-Capitalizacion/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Procesamiento

In [2]:
linux = True
device = None

if linux:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
else:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("usando:", device)

usando: cuda


# Ejericio b)

## Busqueda de fuentes

### Fuente 1: Conjunto de preguntas en espa;ol

In [3]:
questions, question_for_mixture = get_questions()

Se descargaron 5000 preguntas en Español.


### Fuente 2: Dataset provisto para Notebook 10

In [4]:
oraciones_rnn = get_notebook_dataset()

Se descargaron 997 oraciones en Español (del dataset del notebook 10).


### Fuente 3: Dataset sintetico generado con Gemini

In [5]:
oraciones_sinteticas = get_gemini_dataset()

Hay 1413 oraciones sintéticas.


### Fuente 4: Articulos de Wikipedia

In [6]:
frases_wikipedia = get_wikipedia_dataset()

['Argentina, oficialmente República Argentina,[a]\u200b es un país soberano de América del Sur, ubicado en el extremo sur y sudeste de ese subcontinente.', 'Adopta la forma de gobierno republicana, democrática, representativa y federal.', 'Poseen Carta Magna, bandera y fuerzas de seguridad propias, el dominio de los recursos naturales circunscriptos en su territorio y delegan los poderes exclusivos al Gobierno Federal.', 'Hasta mediados del siglo XX, fue una de las economías más prósperas del mundo.', 'No obstante, es la segunda economía más importante de Sudamérica —detrás de Brasil— y la 24.º más grande del mundo por PIB nominal.']


### Fuente 5: Subtitulos de peliculas

In [7]:
esperando_la_carroza, frases_relatos_salvajes = get_pelis_dataset()

✅ Se extrajeron 947 frases completas y se guardaron en 'dialogos_esperando_la_carroza.json'
✅ Frases extraídas y guardadas. Total: 947
['¿Dónde se me ve nerviosa? Jorge ocúpate si se ensució.', '¿Verdad? Forense.- Una vez que el juez dé el permiso, podrán llevarla a...', '_ Elvira, no quisiera poner el dedo en la llaga, pero al fin y al cabo Sergio y vos están viviendo en esta casa que fue de Mamá C ora, y estos son sus muebles.', 'es que los muertos me impresionan.', 'Yo te lo plancho, Susana.', '¿Qué no le va a poder dar?', 'Por ahí.', '¿Para qué querés saber con quién se acostó Elvira?', 'Yo sirvo, chiquita.', 'Lo que le vendría bien es que te la llevaras a vivir un tiempo a tu casa.']
✅ Se extrajeron 1000 frases de Relatos Salvajes.


### Fuente 6 (beta): Mixture de oraciones

In [8]:
cant_oraciones = len(oraciones_sinteticas)
question_for_mixture = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", q).strip()) for q in question_for_mixture]
oraciones_sinteticas = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", a).strip()) for a in oraciones_sinteticas]

tanda_1 = question_for_mixture[:cant_oraciones]
question_affirmation = [f"{q} {a}" for q, a in zip(tanda_1, oraciones_sinteticas)]

tanda_2 = question_for_mixture[cant_oraciones:2*cant_oraciones]
affirmation_question = [f"{a} {q}" for q, a in zip(tanda_2, oraciones_sinteticas)]

tanda_3 = question_for_mixture[2*cant_oraciones:3*cant_oraciones]
tanda_3_shuffled = random.sample(tanda_3, len(tanda_3))
question_question = [f"{q} {p}" for q, p in zip(tanda_3, tanda_3_shuffled)]

mixtures = question_affirmation + affirmation_question + question_question

random.sample(mixtures, 5)


['Coca-Cola produce más de 500 marcas. ¿Cuál es el color de fondo de la enseña de Haro?',
 '¿Qué magnitud alcanzó el seísmo ocurrido hace una semana? ¿Quién hizo acto de presencia por el barrio de Sant Ramon?',
 '¿Cuál es la empresa editora del manga de Naruto? ¿Quién fundó el Archivo General de Simancas?',
 '¿Quién les está haciendo la vida imposible? La sede de Johnson & Johnson está en América.',
 'Qué pena que el concierto se haya cancelado en el último momento ¿Cuándo llegó Wollstonecraft a París?']

## Juntamos las fuentes

In [9]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes + mixtures

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))
print('Cantidad de oraciones Compuestas:',len(mixtures))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Cantidad total de oraciones: 20244
Cantidad de oraciones de preguntas: 5000
Cantidad de oraciones en espa;ol de hugging face: 997
Cantidad de oraciones sintéticas: 1413
Cantidad de oraciones de Wikipedia: 6648
Cantidad de oraciones de Esperando la carroza: 947
Cantidad de oraciones de Relatos Salvajes: 1000
Cantidad de oraciones Compuestas: 4239
Algunas oraciones aleatorias:


['¿De qué ciudad viene el Ivesur-PTV? ¿Cuál era la actitud del Círculo Beliáyev respecto a las costumbres compositivas de Occidente?',
 '== Autores más relevantes ==\n\n\n== Subsidiarias de DC Comics (propiedad de Warner Bros.',
 'Argentina, en años sin campeonatos mundiales por la guerra en Europa, tuvo absoluta supremacía en el fútbol continental.',
 'La reunión de la OEA no llegó a un consenso sobre la situación.',
 'El concepto de democracia participativa propone la creación de formas democráticas directas para atenuar el carácter puramente representativo (audiencias públicas, recursos administrativos, ombudsman).']

Separamos en conjuntos de `train` y `test` con el tokenizer de `BERT`

In [10]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)

print(len(train_sents))
print(len(test_sents))

19231
1013


## Importamos el modelo

### Sin atencion

In [11]:
from train.RNNBidirectional import PunctuationCapitalizationRNNBidirectional

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

bert_embeddings = bert_model.embeddings.word_embeddings
for param in bert_model.parameters():
    param.requires_grad = False

N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in bert_model.pooler.parameters():
    param.requires_grad = True


model = PunctuationCapitalizationRNNBidirectional(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)

ckpt = torch.load("model_bidirec.pt", map_location=device)
# si guardaste state_dict puro
if isinstance(ckpt, dict) and "model_state_dict" not in ckpt:
    model.load_state_dict(ckpt)

# si guardaste un dict con más cosas (epoch, optim, etc.)
elif "model_state_dict" in ckpt:
    model.load_state_dict(ckpt["model_state_dict"])

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Total parameters: 180,944,905
Trainable parameters: 17,857,801


In [ ]:
from sklearn.metrics import classification_report

def evaluate_model(dataloader, model, label_index, target_names, task_name):
    y_true, y_pred = [], []

    for batch in dataloader:
        input_ids, attention_mask, labels_punct, labels_cap = batch
        labels = labels_punct if label_index == 0 else labels_cap

        with torch.no_grad():
            logits_punct, logits_cap = model(input_ids, attention_mask)

        logits = logits_punct if label_index == 0 else logits_cap
        preds = torch.argmax(logits, dim=-1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

    flattened_y_true = [label for seq in y_true for label in seq]
    flattened_y_pred = [pred for seq in y_pred for pred in seq]

    filtered_y_true = [
        label for label, pred in zip(flattened_y_true, flattened_y_pred) if label != -100 and pred != -100
    ]
    filtered_y_pred = [
        pred for label, pred in zip(flattened_y_true, flattened_y_pred) if label != -100 and pred != -100
    ]

    print(f"\nClassification Report for {task_name}:")
    print(classification_report(filtered_y_true, filtered_y_pred, target_names=target_names))


evaluate_model(dataloader_test, model, label_index=0, target_names=PUNCT_TAGS, task_name="Punctuation Prediction")
evaluate_model(dataloader_test, model, label_index=1, target_names=CAP_TAGS, task_name="Capitalization Prediction")



Classification Report for Punctuation Prediction:
              precision    recall  f1-score   support

           Ø       0.99      0.98      0.98     19951
           ,       0.52      0.85      0.64       499
           .       0.83      0.76      0.79       652
           ?       0.84      0.70      0.76       565
           ¿       0.93      0.85      0.89       579

    accuracy                           0.96     22246
   macro avg       0.82      0.83      0.81     22246
weighted avg       0.97      0.96      0.96     22246


Classification Report for Capitalization Prediction:
              precision    recall  f1-score   support

       lower       0.98      0.99      0.99     15161
        init       0.96      0.95      0.95      4734
         mix       0.90      0.66      0.76        65
       upper       0.87      0.95      0.91       260

    accuracy                           0.98     20220
   macro avg       0.93      0.89      0.90     20220
weighted avg       0.98   

### Con atencion

In [11]:
from train.RNNBidirectionalAttention import PunctuationCapitalizationRNNBidirectionalAttention 

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

bert_embeddings = bert_model.embeddings.word_embeddings
for param in bert_model.parameters():
    param.requires_grad = False

N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in bert_model.pooler.parameters():
    param.requires_grad = True


model = PunctuationCapitalizationRNNBidirectionalAttention(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)



total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Total parameters: 181,995,529
Trainable parameters: 18,908,425


In [12]:
punct_weights_tensor, cap_weights_tensor = compute_class_weights(
    dataloader_train,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS),
    device=device,
    beta=0.7
)

criterion_punct = nn.CrossEntropyLoss(ignore_index=-100, weight=punct_weights_tensor)
criterion_cap   = nn.CrossEntropyLoss(ignore_index=-100, weight=cap_weights_tensor)

"""
trainable_params = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.rnn.parameters()) \
  + list(model.punct_classifier.parameters()) \
  + list(model.cap_classifier.parameters())
"""

trainable_params = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.lstm1.parameters()) \
  + list(model.lstm2.parameters()) \
  + list(model.attention.parameters()) \
  + list(model.punct_classifier.parameters()) \
  + list(model.cap_classifier.parameters())

optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)

train(model, dataloader_train=dataloader_train, dataloader_test=dataloader_test,optimizer=optimizer, criterion_punct=criterion_punct, criterion_cap = criterion_cap, device=device, epochs=20)

Epoch 1 | Train Loss: 1.9828
Epoch 2 | Train Loss: 1.2857
Epoch 3 | Train Loss: 0.8370
Epoch 4 | Train Loss: 0.6504
Epoch 5 | Train Loss: 0.5590
Epoch 6 | Train Loss: 0.5034
Epoch 7 | Train Loss: 0.4591
Epoch 8 | Train Loss: 0.4270
Epoch 9 | Train Loss: 0.4025
Epoch 10 | Train Loss: 0.3850
Epoch 11 | Train Loss: 0.3628
Epoch 12 | Train Loss: 0.3470
Epoch 13 | Train Loss: 0.3322
Epoch 14 | Train Loss: 0.3163
Epoch 15 | Train Loss: 0.3061
Epoch 16 | Train Loss: 0.2918
Epoch 17 | Train Loss: 0.2832
Epoch 18 | Train Loss: 0.2715
Epoch 19 | Train Loss: 0.2644
Epoch 20 | Train Loss: 0.2529


## Evaluacion

In [ ]:
evaluate(model, dataloader_test, device)

In [29]:
entrada = "es terrible lo que está pasando en chaco te enteraste"
print(f"{predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

Es terrible, lo que está pasando, en Chaco, te enteraste.


## Export modelo

In [14]:
torch.save(model.state_dict(), "model_bidirec_attention.pt")